In [32]:
import os
from sqlalchemy import create_engine, text

engine = create_engine(os.environ("DATABASE_URL"))
conn = engine.connect()

res = conn.execute(text("SELECT now()")).fetchall()
print(res)

[(datetime.datetime(2024, 3, 2, 19, 58, 49, 144831, tzinfo=datetime.timezone.utc),)]


In [10]:
import pandas as pd
import os
from googleapiclient.discovery import build
import isodate
from sqlalchemy import create_engine,text

### YouTube API credentials

In [11]:
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")

In [12]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=YOUTUBE_API_KEY)

### Import channel IDs from CSV

In [14]:
# File path for CSV import
csv_file_path = 'channels/channel_ids.csv'

# Read channel IDs from CSV file into a list
import_channel_df = pd.read_csv(csv_file_path, header=None, names=["channel_id"])

In [15]:
import_channel_df

,channel_id
0,UCtYLUTtgS3k1Fg4y5tAhLbw
1,UC4JX40jDee_tINbkjycV4Sg
2,UC8butISFwT-Wl7EV0hUK0BQ
3,UCFbNIlppjAuEX4znoulh0Cw
4,UC6AVa0vSrCpuskzGDDKz_EQ
...,...
96,UCH6gDteHtH4hg3o2343iObA
97,UC3iyLcDyYxk2nN7M5DsYaYQ
98,UC4xKdmAXFh4ACyhpiQ_3qBw
99,UCtC_WTVuo9k3Zol0ZB6u5mQ


In [16]:
def get_channel_stats(youtube, channel_ids):
    all_data = []

    try:
        # Split channel_ids list into chunks of up to 50 ids each
        id_chunks = [channel_ids[i:i+50] for i in range(0, len(channel_ids), 50)]

        for id_chunk in id_chunks:
            request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(id_chunk)
            )
            response = request.execute()

            for item in response.get('items', []):
                data = {
                    'channel_name': item['snippet']['title'],
                    'channel_id': item['id'],
                    'description': item['snippet']['description'],
                    'subscriber_count': item['statistics']['subscriberCount'],
                    'view_count': item['statistics']['viewCount'],
                    'video_count': item['statistics']['videoCount'],
                    'playlist_id': item['contentDetails']['relatedPlaylists']['uploads'],
                    'start_date': item['snippet']['publishedAt'],
                    'country': item['snippet'].get('country', None),
                }
                all_data.append(data)
    except Exception as e:
        print(f"Error occurred: {e}")

    return pd.DataFrame(all_data)

In [17]:
channels_df = get_channel_stats(youtube, import_channel_df['channel_id'])

In [18]:
channels_df

,channel_name,channel_id,description,subscriber_count,view_count,video_count,playlist_id,start_date,country
0,Yosh,UCh1zLfuN6F_X4eoNKCsyICA,Support me on Patreon: https://patreon.com/yos...,117000,23709138,9,UUh1zLfuN6F_X4eoNKCsyICA,2018-12-13T01:16:04Z,None
1,Keith Galli,UCq6XkhO5SZ66N04IcPbqNcw,Recent MIT Graduate. I make educational videos...,212000,14647613,82,UUq6XkhO5SZ66N04IcPbqNcw,2016-12-27T20:18:51Z,US
2,Alex The Analyst,UC7cs8q-gJRlGwj4A8OmCmXg,My name is Alex Freberg and on this channel I ...,715000,30841698,289,UU7cs8q-gJRlGwj4A8OmCmXg,2020-01-08T05:04:24.970712Z,US
3,Jay Feng,UCcQx1UnmorvmSEZef4X7-6g,"Learn about full stack data science, data scie...",41000,2348514,245,UUcQx1UnmorvmSEZef4X7-6g,2019-11-19T19:16:30.516571Z,US
4,Steve (Builder.io),UCGmR6lhKMlCkvrvb39vPtdA,CEO @ Builder.io - the drag & drop headless CM...,85000,5588330,152,UUGmR6lhKMlCkvrvb39vPtdA,2008-08-03T20:23:48Z,None
...,...,...,...,...,...,...,...,...,...
96,ProjectPro - Data Science Projects,UCfuyYAtTG15_db2PRuaerZg,Welcome to the ProjectPro Youtube channel!\nWe...,9880,631662,750,UUfuyYAtTG15_db2PRuaerZg,2020-05-20T06:22:20.335331Z,IN
97,Mısra Turp,UCpNUYWW0kiqyh0j5Qy3aU7w,Here is where we learn! This is a space to tak...,29900,1204960,143,UUpNUYWW0kiqyh0j5Qy3aU7w,2020-03-22T15:13:05.584193Z,NL
98,Python Simplified,UCKQdc0-Targ4nDIAUrlfKiA,Hi everyone! My name is Mariya and I'm a softw...,219000,9036809,145,UUKQdc0-Targ4nDIAUrlfKiA,2019-01-25T00:48:09Z,US
99,Emergent Garden,UCwBhBDsqiQflTMLy2epbQVw,"Life simulations, neural networks, cellular au...",99600,4130888,31,UUwBhBDsqiQflTMLy2epbQVw,2020-08-03T19:15:03.446608Z,None


In [19]:
from datetime import datetime
channels_df['etl_date'] = datetime.today().strftime('%Y-%m-%d')

In [20]:
channels_df

,channel_name,channel_id,description,subscriber_count,view_count,video_count,playlist_id,start_date,country,etl_date
0,Yosh,UCh1zLfuN6F_X4eoNKCsyICA,Support me on Patreon: https://patreon.com/yos...,117000,23709138,9,UUh1zLfuN6F_X4eoNKCsyICA,2018-12-13T01:16:04Z,None,2024-03-02
1,Keith Galli,UCq6XkhO5SZ66N04IcPbqNcw,Recent MIT Graduate. I make educational videos...,212000,14647613,82,UUq6XkhO5SZ66N04IcPbqNcw,2016-12-27T20:18:51Z,US,2024-03-02
2,Alex The Analyst,UC7cs8q-gJRlGwj4A8OmCmXg,My name is Alex Freberg and on this channel I ...,715000,30841698,289,UU7cs8q-gJRlGwj4A8OmCmXg,2020-01-08T05:04:24.970712Z,US,2024-03-02
3,Jay Feng,UCcQx1UnmorvmSEZef4X7-6g,"Learn about full stack data science, data scie...",41000,2348514,245,UUcQx1UnmorvmSEZef4X7-6g,2019-11-19T19:16:30.516571Z,US,2024-03-02
4,Steve (Builder.io),UCGmR6lhKMlCkvrvb39vPtdA,CEO @ Builder.io - the drag & drop headless CM...,85000,5588330,152,UUGmR6lhKMlCkvrvb39vPtdA,2008-08-03T20:23:48Z,None,2024-03-02
...,...,...,...,...,...,...,...,...,...,...
96,ProjectPro - Data Science Projects,UCfuyYAtTG15_db2PRuaerZg,Welcome to the ProjectPro Youtube channel!\nWe...,9880,631662,750,UUfuyYAtTG15_db2PRuaerZg,2020-05-20T06:22:20.335331Z,IN,2024-03-02
97,Mısra Turp,UCpNUYWW0kiqyh0j5Qy3aU7w,Here is where we learn! This is a space to tak...,29900,1204960,143,UUpNUYWW0kiqyh0j5Qy3aU7w,2020-03-22T15:13:05.584193Z,NL,2024-03-02
98,Python Simplified,UCKQdc0-Targ4nDIAUrlfKiA,Hi everyone! My name is Mariya and I'm a softw...,219000,9036809,145,UUKQdc0-Targ4nDIAUrlfKiA,2019-01-25T00:48:09Z,US,2024-03-02
99,Emergent Garden,UCwBhBDsqiQflTMLy2epbQVw,"Life simulations, neural networks, cellular au...",99600,4130888,31,UUwBhBDsqiQflTMLy2epbQVw,2020-08-03T19:15:03.446608Z,None,2024-03-02


In [25]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime
metadata_obj = MetaData()

channel_stats_daily_table = Table(
   'channels_daily',
   metadata_obj,
   Column('channel_id', String, primary_key=True),
   Column('etl_date', DateTime, primary_key=True),
   Column('channel_name', String),
   Column('subscriber_count', Integer),
   Column('view_count', Integer),
   Column('video_count', Integer),
)
metadata_obj.create_all(engine)

In [26]:
# Push df to database
channels_df.to_sql(name="channels_daily", con=engine, if_exists='append', index=False)

101

In [33]:
# Test database with simple query
query = text('SELECT * FROM channels_daily')
test_df = pd.read_sql_query(query, conn)

In [34]:
test_df

,channel_name,channel_id,description,subscriber_count,view_count,video_count,playlist_id,start_date,country,etl_date
0,Yosh,UCh1zLfuN6F_X4eoNKCsyICA,Support me on Patreon: https://patreon.com/yos...,117000,23709138,9,UUh1zLfuN6F_X4eoNKCsyICA,2018-12-13T01:16:04Z,None,2024-03-02
1,Keith Galli,UCq6XkhO5SZ66N04IcPbqNcw,Recent MIT Graduate. I make educational videos...,212000,14647613,82,UUq6XkhO5SZ66N04IcPbqNcw,2016-12-27T20:18:51Z,US,2024-03-02
2,Alex The Analyst,UC7cs8q-gJRlGwj4A8OmCmXg,My name is Alex Freberg and on this channel I ...,715000,30841698,289,UU7cs8q-gJRlGwj4A8OmCmXg,2020-01-08T05:04:24.970712Z,US,2024-03-02
3,Jay Feng,UCcQx1UnmorvmSEZef4X7-6g,"Learn about full stack data science, data scie...",41000,2348514,245,UUcQx1UnmorvmSEZef4X7-6g,2019-11-19T19:16:30.516571Z,US,2024-03-02
4,Steve (Builder.io),UCGmR6lhKMlCkvrvb39vPtdA,CEO @ Builder.io - the drag & drop headless CM...,85000,5588330,152,UUGmR6lhKMlCkvrvb39vPtdA,2008-08-03T20:23:48Z,None,2024-03-02
...,...,...,...,...,...,...,...,...,...,...
197,ProjectPro - Data Science Projects,UCfuyYAtTG15_db2PRuaerZg,Welcome to the ProjectPro Youtube channel!\nWe...,9880,631662,750,UUfuyYAtTG15_db2PRuaerZg,2020-05-20T06:22:20.335331Z,IN,2024-03-02
198,Mısra Turp,UCpNUYWW0kiqyh0j5Qy3aU7w,Here is where we learn! This is a space to tak...,29900,1204960,143,UUpNUYWW0kiqyh0j5Qy3aU7w,2020-03-22T15:13:05.584193Z,NL,2024-03-02
199,Python Simplified,UCKQdc0-Targ4nDIAUrlfKiA,Hi everyone! My name is Mariya and I'm a softw...,219000,9036809,145,UUKQdc0-Targ4nDIAUrlfKiA,2019-01-25T00:48:09Z,US,2024-03-02
200,Emergent Garden,UCwBhBDsqiQflTMLy2epbQVw,"Life simulations, neural networks, cellular au...",99600,4130888,31,UUwBhBDsqiQflTMLy2epbQVw,2020-08-03T19:15:03.446608Z,None,2024-03-02


In [31]:
# Close the connection
conn.close()
engine.dispose()